In [ ]:
import pandas as pd
import csv
import numpy as np
import time 

In [ ]:
#create dataframe from input file
df = pd.DataFrame.from_csv('D:/Nanohub/results/GIS_2011_core_neigh_child/cores_neigh_jacc_2011.csv', sep =',', header = None, index_col = False)
df.columns = ['latlon','latcomlon','doy','parent_id','ip','child_id','cluster_id']


uniq_latlon = df['latlon'].unique().tolist()
uniq_childID = df['child_id'].unique().tolist()
uniq_latcomlon = df['latcomlon'].unique().tolist()
uniq_IP = df['ip'].unique().tolist()

In [ ]:
#create Jaccard matrix
jacc_sims = np.zeros((len(uniq_childID),len(uniq_latlon)))
for i in range(len(uniq_latlon)):
    curr =  df[df['latlon'] == uniq_latlon[i]]
    #print(curr)
    for j in range(len(uniq_childID)):
        if( uniq_childID[j] in curr['child_id'].values):
            #print(j,i)
            jacc_sims[j][i] = 1

In [ ]:
#print info
print('number of unique resources in 2011:',len(uniq_childID))
print('number of unique locations in 2011:',len(uniq_latlon))
print('dimensions of similarity matrix:',jacc_sims.shape)

In [ ]:
#test to see if matrix got initialized
jacc_sims[59]

In [ ]:
#incorrect
#Jaccard similarity using Scikit learn seems to be giving incorrect results
f = open('D:/Nanohub/results/GIS_2011_core_neigh_child/sim_jacc_row.csv','w')
writer = csv.writer(f,lineterminator = '\n')

from sklearn.metrics import jaccard_similarity_score
for j in range(len(jacc_sims)):
    simres = []
    for i in range(len(jacc_sims)):
        jacc_sim = jaccard_similarity_score(jacc_sims[j], jacc_sims[i])
        if jacc_sim > 0.8 and i!=j:
            #print(uniq_childID[i], jacc_sim)
            simres.append(uniq_childID[i])
    writer.writerow(simres)
f.close()


In [ ]:
#direct implementation of Jaccard index
f = open('D:/Nanohub/results/GIS_2011_core_neigh_child/sim_jacc_2011.csv','w')
writer = csv.writer(f,lineterminator = '\n')
for j in range(len(jacc_sims)):
    res1 = jacc_sims[j]
    
    for i in range(len(jacc_sims)):
        all_1 = 0
        union_1 = 0
        GPS = []
        if i!=j:
            res2 = jacc_sims[i]
            #print(res1,res2)
            for k in range(len(res2)):
                if res1[k] == res2[k] and res1[k] == 1:
                    union_1 += 2
                    GPS.append(uniq_latcomlon[k])
                all_1 = all_1 + res1[k] + res2[k]
            jacc_sim = float(union_1/all_1)
            if jacc_sim> 0.8:
                tup = ( uniq_childID[j],uniq_childID[i], jacc_sim, GPS)
                writer.writerow(tup)